# spaCy, wersja NASK
7 kwietnia 2022

Model oparty o Herberta w wersji base (https://huggingface.co/allegro/herbert-base-cased).
Trenowany na NKJP 1M po konwersji do UD.

#### Ładowanie modelu, wersja spacy, i wyniki.

In [ ]:
import spacy


nlp = spacy.load("pl_nask")


In [ ]:
print("spaCy version:", spacy.__version__)
performance_data = nlp.meta["performance"]
acc_keys = ["tag_acc", "pos_acc", "morph_acc", "dep_uas", "dep_las", "ents_f", "ents_r", "ents_p"]
print("\nEvaluation results:")
for key in acc_keys:
    print(key, round(performance_data[key] * 100, 2))

In [ ]:
performance_data

#### Skład pipeline'u:

In [ ]:
for name, component in nlp.pipeline:
    print(name, component)

#### Anotacja morfologiczna i leksykalna

Informacja o tagach morfologicznych wygląda nastepująco:

    tok.tag_ 
zawiera pełen tag morfologiczny w tagsecie SGJP

    tok.pos_
zawiera klasę części mowy w tagsecie UPOS

    tok.morph
zawiera cechy morfologiczne w tagsecie UFEATS

    tok.lemma_
zawiera lemat pochodzący z analiz Morfeusza, dezambiguowanych przez tok.tag_ oraz dane frekwencyjne.

In [ ]:
import pandas

def table_tags(doc):
    tok_dicts = []
    for tok in doc:
        tok_dict = {
                    "orth": tok.orth_,
                    "lemma": tok.lemma_,
                    "UPOS": tok.pos_,
                    "SGJP": tok.tag_,}
        tok_dicts.append(tok_dict)
    return pandas.DataFrame(tok_dicts)

def table_morphs(doc):
    tok_dicts = []
    for tok in doc:
        tok_dict = {
                    "orth": tok.orth_,
                    "lemma": tok.lemma_,
                    "UFEAT": str(tok.morph)[:40]}
        tok_dicts.append(tok_dict)
    return pandas.DataFrame(tok_dicts)

In [ ]:
txt = "Ani skuteczna dyplomacja, ani moralne wsparcie, ani profetyczne gesty. \
Tak podsumować można linię Stolicy Apostolskiej w czasie wojny w Ukrainie. \
Rosyjska agresja boleśnie zweryfikowała teologię pokoju i wizję geopolityczną papieża."
doc = nlp(txt)
print(table_tags(doc), "\n\n")
print(table_morphs(doc), "\n\n")

Obsługujemy prostą, słownikową dedyminutywizację: Z SGJP i Wiktionary zebraliśmy ponad 6 tys. par zdrobnienie - forma bazowa, które wykorzystujemy, słownik ten można rozbudowywać, znajduje się w plikach modelu.

    tok._.is_diminutive
wartość logiczna opisująca czy dany token wyraża zdrobnienie

    tok._.diminutive_chain
lista zdrobnień w łańcuchu aż do ostatniej formy

In [ ]:
txt = "Spotkałem się z Alą, żeby porozmawiać o jej milutkim synku - Piotrusiu. Ona jest wspaniałą mamusią."
doc2 = nlp(txt)
for tok in doc2:
    print(tok.i, tok.orth_, tok.lemma_, tok._.is_diminutive, tok._.diminutive_chain)


Morfeusz zwraca również kilka kwalifikatorów o charakterze leksykalnym, zapisujemy je w rozszerzonych atrybutach:

    tok._.properness
lista kwalifikatorów opisujących pospolitość/własność rzeczowników.

    tok._.disambiguator
"rozpodabniacz", rozróżniający różne leksemy, o identycznie brzmiących lematach, ale różnych wzorcach odmiany.

    tok._.style
lista kwalifikatorów dotyczących m.in. nacechowania stylistycznego.

    tok._.is_ign
atrybut określający, czy wskazane słowo jest poza słownikiem morfeusza, atrybut ten można wykorzystywać zamiast tok.is_oov, którego to nie można nadpisać, i jest normalnie ustawiane na podstawie embeddingów word2vec, których w tym modelu nie stosujemy

    tok._.freq
częstość bezwzględna danego lematu w NKJP1M

In [ ]:
def table_additional_annotations(doc):
    tok_dicts = []
    for tok in doc:
        tok_dict = {
                    "orth": tok.orth_,
                    "properness": tok._.properness,
                    "disambiguator": tok._.disambiguator,
                    "style": tok._.style,
                    "ign": tok._.is_ign,
                    "freq": tok._.freq
        }
        tok_dicts.append(tok_dict)
    return pandas.DataFrame(tok_dicts)

txt2 = "A to zwykłe mendy, co im zawiniły firanki?!? Podaj mi francuzy, s'il vous plait"
doc2 = nlp(txt2)

print(table_additional_annotations(doc2))

### Regułowe wyszukiwanie wzorców

(https://spacy.io/usage/rule-based-matching)

SpaCy umożliwia wyszukiwanie w tekście wzorców. Wzorce opisują ciągłe sekwencje tokenów, w terminach udostępnianych przez SpaCy atrybutów tokenów. Liczba obsługiwanych atrybutów oraz ekspresywność warunków jakie możemy formułować przekracza zakres tej prezentacji, natomiast możemy wskazać kilka podstawowych funkcjonalności.

Wzorce mogą być również definiowane w terminach UFEATS, wówczas porównujemy nie pojedyncze wartości, a słowniki wartości.

In [ ]:
from spacy.matcher import Matcher

pattern_fem_sing = [{"MORPH": {"IS_SUPERSET": ["Number=Sing", "Gender=Fem"]}}]

matcher = Matcher(nlp.vocab, validate=True)
matcher.add("FemSing", [pattern_fem_sing])

matches = matcher(doc)
for match in matches:
    rule_identifier, start, end = match
    rule_name = nlp.vocab.strings[rule_identifier]
    print(f"{rule_name}: {doc[start:end]}")

In [ ]:
matcher.remove("FemSing")
pattern_fem_sing_noun = [{"MORPH": {"IS_SUPERSET": ["Number=Sing", "Gender=Fem"]},
                          "POS": "NOUN"}]
matcher.add("FemSingNoun", [pattern_fem_sing_noun])

matches = matcher(doc)
for match in matches:
    rule_identifier, start, end = match
    rule_name = nlp.vocab.strings[rule_identifier]
    print(f"{rule_name}: {doc[start:end]}")

In [ ]:
matcher.remove("FemSingNoun")
pattern_fem_sing_adj_noun = [{"MORPH": {"IS_SUPERSET": ["Number=Sing", "Gender=Fem"]},#Pierwszy token
                          "POS": "ADJ"},
                         {"MORPH": {"IS_SUPERSET": ["Number=Sing", "Gender=Fem"]},# Drugi token
                          "POS": "NOUN"}]
matcher.add("FemSingAdjNoun", [pattern_fem_sing_adj_noun])
pattern_fem_sing_noun_adj = [{"MORPH": {"IS_SUPERSET": ["Number=Sing", "Gender=Fem"]},#Pierwszy token
                          "POS": "NOUN"},
                         {"MORPH": {"IS_SUPERSET": ["Number=Sing", "Gender=Fem"]},# Drugi token
                          "POS": "ADJ"}]
matcher.add("FemSingNounAdj", [pattern_fem_sing_noun_adj])

matches = matcher(doc)
for match in matches:
    rule_identifier, start, end = match
    rule_name = nlp.vocab.strings[rule_identifier]
    print(f"{rule_name}: {doc[start:end]}")

### Parsowanie zależnościowe
Parser wytrenowano na NKJP 1M w wersji UD, zgodnym z anotacją PDB. Dane są częściowo anotowane ręcznie, częściowo automatycznie (COMBO).

In [ ]:
from spacy import displacy
displacy.render(doc, style='dep',jupyter=True)

#### Regułowe wyszukiwanie wzorców w drzewach zależnościowych

(https://spacy.io/usage/rule-based-matching)

W języku polskim wymóg odgórnego określania kolejności tokenów, oraz ciągłości sekwencji może być mocno ograniczający. Możemy więc wykorzystać narzędzie o większej mocy, pozwalające definiować wzorce w terminach położenia w drzewie zależnościowym (które może być zgoła odmienne od horyzontalnego porządku tokenów w tekście).

In [ ]:
from spacy.matcher import DependencyMatcher

matcher = DependencyMatcher(nlp.vocab)

dep_noun_adj = \
[
    {
        "RIGHT_ID": "noun",
        "RIGHT_ATTRS": {"POS": "NOUN"}
    },
    {
        "LEFT_ID": "noun",
        "REL_OP": ">",
        "RIGHT_ID": "modifier",
        "RIGHT_ATTRS": {"DEP": "amod"}
    }
]

matcher.add("NounAdj", [dep_noun_adj])
matches = matcher(doc)
for match in matches:
    rule_identifier, matching_tokens = match
    rule_name = nlp.vocab.strings[rule_identifier]
    ordered_phrase = [doc[tok_i] for tok_i in sorted(matching_tokens)]
    print(f"{rule_name}: {ordered_phrase}")

In [ ]:
coordination = [{
        "RIGHT_ID": "subordinate",
        "RIGHT_ATTRS": {"DEP": "conj"}
    },
    {
        "LEFT_ID": "subordinate",
        "REL_OP": "<",
        "RIGHT_ID": "superior",
        "RIGHT_ATTRS": {}
    },
    {
        "LEFT_ID": "subordinate",
        "REL_OP": ">",
        "RIGHT_ID": "cc",
        "RIGHT_ATTRS": {"DEP": {"IN": ["cc", "cc:preconj"]}}
    }]
    

from spacy.matcher import DependencyMatcher
matcher = DependencyMatcher(nlp.vocab)
matcher.add("Coordination", [coordination])
matches = matcher(doc)
for match in matches:
    rule_identifier, matching_tokens = match
    rule_name = nlp.vocab.strings[rule_identifier]
    ordered_phrase = [doc[tok_i] for tok_i in sorted(matching_tokens)]
    print(f"{rule_name}: {ordered_phrase}")

### Rozpoznawanie jednostek nazewniczych

Model obsługuje 5 podstawowych kategorii opisanych w NKJP:
PLACENAME, GEOGNAME, PERSNAME, TIME, DATE

Nie obsługujemy jednostek zazębiających się (a więc również jednostek zagnieżdżonych).

In [ ]:
displacy.render(doc, style='ent',jupyter=True)

#### Przetwarzanie batchy tekstów.

In [ ]:
orchids ="Storczykowate są rodziną kosmopolityczną, występującą na wszystkich kontynentach, z wyjątkiem Antarktydy. Największe zróżnicowanie gatunkowe storczyków występuje w strefie międzyzwrotnikowej, a zwłaszcza w tropikach na kontynentach amerykańskich i w Azji południowo-wschodniej po Nową Gwineę. W tropikach amerykańskich rośnie 350 rodzajów i ok. 10 tys. gatunków, tylko w Malezji jest ich 4,5 tys. gatunków, a na Nowej Gwinei 2,3 tys. Z tropikami związanych jest 36 najbardziej zróżnicowanych gatunkowo rodzajów, liczących ponad 100 gatunków, i tylko nieliczne z nich mają przedstawicieli poza strefą równikową."
nettles = "Pokrzywa – rodzaj jednorocznych roślin zielnych lub bylin z rodziny pokrzywowatych (Urticaceae Juss.). Należy do niej co najmniej 68 gatunków rozpowszechnionych na całej kuli ziemskiej z wyjątkiem Antarktydy. Rośliny niektórych gatunków dostarczają włókna i są jadalne. "
bridge = "Brydż (ang. bridge) – logiczna gra karciana, w której bierze udział czterech graczy tworzących dwie rywalizujące ze sobą pary[2]. Gracze stanowiący parę siedzą naprzeciwko siebie. Każda para stara się uzyskać lepszy wynik punktowy od wyniku przeciwników. Gra składa się z dwóch odrębnych części: licytacji oraz rozgrywki. Podczas licytacji gracze deklarują wzięcie pewnej minimalnej liczby lew oraz wskazują kolor atutowy lub jego brak, a najwyższa deklaracja staje się kontraktem ostatecznym, z którego trzeba się wywiązać podczas drugiej części zwanej rozgrywką."
jokers = "Joker (wym. „dżoker” czyli ang. żartowniś), dżoker – jedna z kart do gry, w niektórych grach karcianych (na przykład kierki) służy do zastępowania dowolnej innej karty. W standardowej brydżowej talii kart znajdują się dwa lub trzy jokery oprócz 52 kart zwykłych. Najczęściej spotykanym wizerunkiem na jokerze jest kolorowo ubrany błazen (trefniś, ang. joker od joke – „żart”, z łac. iocus czytaj jokus) w czapce z dzwoneczkami. Jokery oprócz wizerunku błazna bywają oznaczane w narożniku karty gwiazdką lub (niekiedy, jeśli nie koliduje to z oznaczeniem waleta) literą J; spotykane są też inne oznaczenia, na przykład znakiem dolara."
tanks = "Pierwsze czołgi brytyjskie przypominały opancerzone skrzynie, opasane z dwóch stron metalowymi gąsienicami. Nowy rodzaj mechanizmu jezdnego umożliwiał pokonywanie trudnych przeszkód, w tym okopów, a także miażdżenie zasieków z drutu kolczastego. Pierwsze czołgi były maszynami bardzo prymitywnymi. Aby wykonać ostry skręt, wymagały skoordynowanej pracy czterech osób, co było nie lada osiągnięciem. Pojazd nie miał wentylacji, co powodowało, że gazy spalinowe i prochowe wywoływały często omdlenia i zatrucia załogi."
pistol = "Pistolet – krótka, ręczna broń palna (z wyłączeniem rewolwerów) zasilana najczęściej amunicją pistoletową, rzadziej rewolwerową (słabszą od karabinowej i pośredniej). Pistolety przeznaczone są do walki na krótkim dystansie (do 50 m). Charakteryzują się krótką lufą, małymi gabarytami i chwytem (rękojeścią) przystosowanym do strzelania z jednej ręki. Najpowszechniej stosowane w wojsku, policji i ochronie. Są także popularną bronią sportową. "
pope = "Biskupi Rzymu oparli swój prymat na sukcesji apostolskiej, zgodnie z tradycją, według której pierwszym biskupem Rzymu był Piotr Apostoł, który zginął tam śmiercią męczeńską. Nowy Testament milczy wprawdzie na ten temat i wspomina tylko o podróży św. Piotra do Antiochii (Gal 2, 11), a w zakończeniu Listu do Rzymian Pawła Apostoła pośród licznych osób Piotr nie jest wymieniony, ale o pobycie apostoła w Rzymie mówią inne pisma z pierwszych wieków istnienia chrześcijaństwa, m.in. list biskupa Antiochii Ignacego do Kościoła w Rzymie, napisany za panowania cesarza Trajana (98–117)."
knights = "Wpływ na powstanie tej grupy miały przemiany społeczno-polityczne na terenie dawnego imperium Karolingów. Wiązały się one z kryzysem władzy centralnej i kształtowaniem się stosunków zależności feudalnej. Równocześnie nastąpił wzrost znaczenia ciężkiej konnicy w prowadzeniu wojen, powodujące zapotrzebowanie na konnych wojowników."
george = "We w pełni rozwiniętej wersji zachodniej smok zrobił sobie gniazdo na źródle, którego woda zaopatrywała miasto Silene (prawdopodobnie późniejsza Cyrena w Libii) lub miasto Lod, zależnie od źródeł. W konsekwencji mieszkańcy musieli prosić smoka o opuszczenie gniazda na czas, gdy nabierali wodę. Każdego dnia oferowali mu owcę, a jeśli jej nie znaleźli, musieli oddać zamiast niej jedną dziewczynę. Ofiara była wybierana przez losowanie."
greece = "Grecja pozostaje pod wpływem klimatu śródziemnomorskiego. Cechuje go łagodna zima z suchym, gorącym latem. W najcieplejszym miesiącu średnia temperatura wynosi ponad 22 °C. Są co najmniej cztery miesiące ze średnią temperaturą ponad 10 °C, w zimie mogą zdarzać się przymrozki. Notuje się co najmniej trzy razy więcej opadów atmosferycznych w najwilgotniejszych miesiącach zimowych w porównaniu z suchym latem."
italy = "W epoce żelaza tereny Włoch zamieszkiwali Ligurowie i Sykulowie, a także liczne inne plemiona italskie, celtyckie i iliryjskie. W okresie starożytności tereny Włoch znalazły się pod panowaniem Rzymian. Przed okresem rzymskim tereny Włoch były zamieszkiwane przez Fenicjan i Greków. Od średniowiecza do Risorgimento, pomimo że Półwysep Apeniński był spójny pod względem językowym i kulturowym, jego historia składała się z dziejów niezależnych republik i księstw oraz obcych posiadłości i stref wpływów."

texts = [orchids, nettles, jokers, bridge, tanks, pistol, pope, knights, george, greece, italy]
dox = list(nlp.pipe(texts))



Podobieństwo semantyczne

In [ ]:
labels = ["orchids ", "nettles", "bridge ", "jokers ", "tanks ", "pistol ",
          "pope ", "knights ", "george ", "greece ", "italy "]
sim_matr = []
for x in dox:
    sim_matr.append([])
    for y in dox:
        sim_matr[-1].append(round(x.similarity(y), 3))

import numpy
arr = numpy.array(sim_matr)
doc_sim = pandas.DataFrame(arr, columns=labels, index=labels)
print(doc_sim)

from itertools import product
tokpairs = []
for x_i in range(len(dox)):
    doc_x = dox[x_i]
    for y_i in range(x_i+1, len(dox)):
        doc_y = dox[y_i]
        tok_prod = product(doc_x, doc_y)
        for t1, t2 in tok_prod:
            sim = round(t1.similarity(t2), 3)
            if sim == 1.0:
                continue
            tokpairs.append((t1, t2, sim))

ranking = sorted(tokpairs, key=lambda x: x[2], reverse=True)
        
for x in ranking[:50]:
    print(x)


Dostęp do wektorów dla dokumentów, tokenów, spanów.

In [ ]:
orchid_doc = dox[0]
doc_vec = orchid_doc.vector
tok_vec = orchid_doc[0].vector
span_vec = orchid_doc[:3].vector
sent_vec = list(orchid_doc.sents)[0].vector
print(sent_vec.shape)

### Flexer

Flexer pozwala, w oparciu o Morfeusza, na odmianę pojedynczych wyrazów, a także fraz. Akceptuje więc pojedyncze tokeny, a także ich listy (niekoniecznie ciągłe).

Dostęp do niego odbywa się poprzez komponent "Morfeusz".

In [ ]:
print(orchid_doc)

In [ ]:
morf_component = nlp.get_pipe("morfeusz")
family = orchid_doc[2]
target_morph = "gen:pl"
inflected = morf_component.flex(family, target_morph)
print(f"{family} -({target_morph})-> {inflected}")

phrase = orchid_doc[2:4]
inflected = morf_component.flex(phrase, target_morph)
print(f"{phrase} -({target_morph})-> {inflected}")


In [ ]:
non_contiguous = [orchid_doc[32], orchid_doc[41]]
target_morph = "inst"
inflected = morf_component.flex(non_contiguous, target_morph)
print(f"{non_contiguous} -({target_morph})-> {inflected}")

Algorytm Flexera jest również do wykorzystania w procesie lematyzacji wyrażeń złożonych:

In [ ]:
tanks_doc = dox[4]
print(tanks_doc)

tanks_phrase = tanks_doc[4:13]
lemmatized = morf_component.lemmatize(tanks_phrase)
print("\n")
print(f"{tanks_phrase} -> {lemmatized}")

In [ ]:
ntxt = """Informuję, że moim identyfikatorem podatkowym, którym posługuję się w rozliczeniach podatkowych z Urzędem Skarbowym jest: Numer PESEL 75123202570 (Niepotrzebne skreślić). Adres do wskazania w PIT: ul. Krauthofera 18a/27, 61-203 Poznań.
2. Pośrednik kredytu hipotecznego: Powszechna Kasa Oszczędności Bank Polski Spółka Akcyjna siedzibą w Warszawie przy ul. Puławskiej 15, 02-515 Warszawa, http://www.pkobp.pl; infolinia 800 302 302. 
Prezentowany wykaz dokumentów jest wspólny dla: -_ Powszechnej Kasy Oszczędnościowej Banku Polskiego Spółki Akcyjnej z siedzibą w Warszawie przy ul. Puławskiej 15, 02-515 Warszawa, zwanej dalej „PKO Bank Polski SA”, - PKO Banku Hipotecznego Spółki Akcyjnej z siedzibą w Gdyni przy ul. Jerzego Waszyngtona 17, 81-342 Gdynia, zwanej dalej „PKO Bank Hipoteczny SA”.
\
"

Wskazówki dla lekarza kierującego: 1. w zakresie diagnostyki: brak 2; w zakresie farmakoterapii: zyx,entom; 3. inne: brak; Anna Sosnowska - Specjalista gardła, foniatra 160-288 Poznań, l. Promienista 347. 

"
"""
doc = nlp(ntxt)